In [2]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [3]:
def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

def get_lt(lt_string = "Lt_CA_Boreal_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets(files))

In [4]:
lt = get_lt() # ~ 2.9 GB

In [5]:
prov = fv.ca_prov()

In [ ]:
# lt = lt.sjoin(prov) # Too computationally expensive??
# lt

In [6]:
snap = fv.prep_gdf(date = '20230501AM',layer='perimeter',handle_multi=False,only_lf=False,area_lim=5, year = "2023", path_region = "BOREAL_NRT_3571_DPS")

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:506: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lon'] = gdf_test.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lat'] = gdf_test.centroid.y


In [7]:
snap = snap.sjoin(prov)

In [99]:
test = snap[snap.duration == 0].groupby("prov_name_en").n_pixels.sum()
test.values

array([490, 232, 248,  45,  12,   1,   9,  42,   4,  48, 318,   1])

In [80]:
#iterables = [test.index, ['20230501AM']]

#pd.MultiIndex.from_product(iterables, names=["prov", "t"])

test = pd.DataFrame(test.values, columns=["farea"], index = test.index)
test.transpose()
#test["t"] = '20230501AM'

prov_name_en,Alberta,British Columbia,Manitoba,New Brunswick,Newfoundland and Labrador,Northwest Territories,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan,Yukon
farea,490,232,248,45,12,1,9,42,4,48,318,1


In [76]:
#test.t.astype("datetime64[ns]")


prov_name_en,Alberta,British Columbia,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan,Yukon
farea,436,196,148,45,12,9,39,4,46,202,1
t,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM,20230501AM


In [246]:
### To generate fire stats
date_range = pd.date_range(start = "2023-03-01 12:00:00", end = "2023-09-15 12:00:00", freq="12H")
#date_range_format = datetime.strptime(date_rage, 
date_snap = date_range.strftime("%Y%m%d%p")
date_snap
#nice_dates = date_range.strftime("%Y-%m-%d %p")

Index(['20230301PM', '20230302AM', '20230302PM', '20230303AM', '20230303PM',
       '20230304AM', '20230304PM', '20230305AM', '20230305PM', '20230306AM',
       ...
       '20230911AM', '20230911PM', '20230912AM', '20230912PM', '20230913AM',
       '20230913PM', '20230914AM', '20230914PM', '20230915AM', '20230915PM'],
      dtype='object', length=397)

In [191]:
#snap["fireID"] = snap.index
ids = snap.groupby("prov_name_en").fireID.unique()

ids

iterables = [ date_range, ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Nova Scotia',
       'Ontario', 'Prince Edward Island', 'Quebec', 'Northwest Territories', 'Saskatchewan', 'Yukon']]
index = pd.MultiIndex.from_product(iterables, names=["t", "prov"])

In [192]:
index

MultiIndex([('2023-05-01 12:00:00',                   'Alberta'),
            ('2023-05-01 12:00:00',          'British Columbia'),
            ('2023-05-01 12:00:00',                  'Manitoba'),
            ('2023-05-01 12:00:00',             'New Brunswick'),
            ('2023-05-01 12:00:00', 'Newfoundland and Labrador'),
            ('2023-05-01 12:00:00',               'Nova Scotia'),
            ('2023-05-01 12:00:00',                   'Ontario'),
            ('2023-05-01 12:00:00',      'Prince Edward Island'),
            ('2023-05-01 12:00:00',                    'Quebec'),
            ('2023-05-01 12:00:00',     'Northwest Territories'),
            ('2023-05-01 12:00:00',              'Saskatchewan'),
            ('2023-05-01 12:00:00',                     'Yukon'),
            ('2023-05-02 00:00:00',                   'Alberta'),
            ('2023-05-02 00:00:00',          'British Columbia'),
            ('2023-05-02 00:00:00',                  'Manitoba'),
          

In [259]:
from datetime import date
str(date.today().strftime("%Y%m%d"))

'20231024'

In [266]:
fire_stats = pd.DataFrame(data = [], columns=["prov_name_en", "t", "farea", "numfires", "num_active_fires",  "tot_frp", "mean_frp", "igs", "num_active_pixels", "fireID"])
snap_list = []
for d,n in zip(date_snap, range(0, len(date_range))):
    print(d)
    ## Read in 12 hour time point
    snap = fv.prep_gdf(date = d,layer='perimeter',handle_multi=False,only_lf=False,area_lim=5, year = "2023", path_region = "BOREAL_NRT_3571_DPS")
    snap["fireID"] = snap.index
    snap = snap.sjoin(prov)
    
    ## Get DF of province stats
    fireIDs = snap.groupby(["prov_name_en", "t"]).fireID.unique().reset_index()
    igs = snap[snap.duration == 0].groupby(["prov_name_en", "t"]).mergeid.count().reset_index()
    igs = igs.rename(columns={"mergeid":"igs"})
    
    num_active_pixels = snap[snap.duration == 0].groupby(["prov_name_en", "t"]).n_pixels.sum() + snap.groupby(["prov_name_en", "t"]).n_newpixels.sum()
    num_active_pixels = num_active_pixels.reset_index()
    num_active_pixels = num_active_pixels.rename(columns={0:"num_active_pixels"})
    
    farea = snap.groupby(["prov_name_en", "t"]).farea.sum().reset_index()
    numfires = snap.groupby(["prov_name_en", "t"]).mergeid.count().reset_index()
    numfires = numfires.rename(columns={"mergeid":"numfires"})
    
    tot_frp = snap.groupby(["prov_name_en", "t"]).meanFRP.sum().reset_index()
    tot_frp = tot_frp.rename(columns={"meanFRP":"tot_frp"})
    
    
    mean_frp = snap.groupby(["prov_name_en", "t"]).meanFRP.mean().reset_index()
    mean_frp = mean_frp.rename(columns={"meanFRP":"mean_frp"})
    
    
    ## Merge the dfs together
    mrg = fireIDs.merge(igs, on = ['prov_name_en', 't'], how = 'outer')
    mrg = mrg.merge(num_active_pixels, on = ['prov_name_en', 't'], how = 'outer')
    mrg = mrg.merge(farea, on = ['prov_name_en', 't'], how = 'outer')
    mrg = mrg.merge(numfires, on = ['prov_name_en', 't'], how = 'outer')
    mrg = mrg.merge(tot_frp, on = ['prov_name_en', 't'], how = 'outer')
    mrg = mrg.merge(mean_frp, on = ['prov_name_en', 't'], how = 'outer')
    
    #snap_list.append(fireIDs)
    fire_stats = pd.concat([fire_stats, mrg])
    
    print(n)
    if((n%5 == 0) | (n == (len(date_snap) -1))):
        print(n / (len(date_snap) -1))

        fire_stats.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats/"+"boreal_snapstats_"+ str(date.today().strftime("%Y%m%d")) +".csv")
    #snap = snap[snap.t ]
#fire_stats = pd.DataFrame(data = snap_list, columns = [ "prov_name_en", "t", "fireIDs"])

20230301PM


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:506: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lon'] = gdf_test.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lat'] = gdf_test.centroid.y


0
0.0
20230302AM


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:506: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lon'] = gdf_test.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lat'] = gdf_test.centroid.y


1
20230302PM


...

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:506: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lon'] = gdf_test.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lat'] = gdf_test.centroid.y


182
20230601AM


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:506: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lon'] = gdf_test.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_test['lat'] = gdf_test.centroid.y


183
20230601PM
could not load file /projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Snapshot/20230601PM


AttributeError: 'NoneType' object has no attribute 'set_index'

In [268]:
fire_stats.num_active_fires.unique()

array([nan], dtype=object)

MultiIndex([('20230501PM',                   'Alberta'),
            ('20230501PM',          'British Columbia'),
            ('20230501PM',                  'Manitoba'),
            ('20230501PM',             'New Brunswick'),
            ('20230501PM', 'Newfoundland and Labrador'),
            ('20230501PM',               'Nova Scotia'),
            ('20230501PM',                   'Ontario'),
            ('20230501PM',      'Prince Edward Island'),
            ('20230501PM',                    'Quebec'),
            ('20230501PM',     'Northwest Territories'),
            ('20230501PM',              'Saskatchewan'),
            ('20230501PM',                     'Yukon')],
           names=['t', 'prov'])

farea
t          prov                           
20230501PM Alberta                     NaN
           British Columbia            NaN
           Manitoba                    NaN
           New Brunswick               NaN
           Newfoundland and Labrador   NaN
           Nova Scotia                 NaN
           Ontario                     NaN
           Prince Edward Island        NaN
           Quebec                      NaN
           Northwest Territories       NaN
           Saskatchewan                NaN
           Yukon                       NaN